![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [37]:
# Start your code here!
import pandas as pd

In [38]:
# Load the datasets
dataset1 = pd.read_csv('women_results.csv')
dataset2 = pd.read_csv('men_results.csv')

# Display the first few rows of each dataset to verify loading
dataset1.head(), dataset2.head()

(   Unnamed: 0        date home_team  ... home_score  away_score        tournament
 0           0  1969-11-01     Italy  ...          1           0              Euro
 1           1  1969-11-01   Denmark  ...          4           3              Euro
 2           2  1969-11-02   England  ...          2           0              Euro
 3           3  1969-11-02     Italy  ...          3           1              Euro
 4           4  1975-08-25  Thailand  ...          3           2  AFC Championship
 
 [5 rows x 7 columns],
    Unnamed: 0        date home_team  ... home_score  away_score  tournament
 0           0  1872-11-30  Scotland  ...          0           0    Friendly
 1           1  1873-03-08   England  ...          4           2    Friendly
 2           2  1874-03-07  Scotland  ...          2           1    Friendly
 3           3  1875-03-06   England  ...          2           2    Friendly
 4           4  1876-03-04  Scotland  ...          3           0    Friendly
 
 [5 rows x 7 

In [39]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4884 non-null   int64 
 1   date        4884 non-null   object
 2   home_team   4884 non-null   object
 3   away_team   4884 non-null   object
 4   home_score  4884 non-null   int64 
 5   away_score  4884 non-null   int64 
 6   tournament  4884 non-null   object
dtypes: int64(3), object(4)
memory usage: 267.2+ KB


In [40]:
dataset1['date'] = pd.to_datetime(dataset1['date'])
dataset2['date'] = pd.to_datetime(dataset2['date'])
start = '2002-01-01'
women = dataset1[(dataset1['date'] >= start) & (dataset1['tournament'] == 'FIFA World Cup')]
men = dataset2[(dataset2['date'] >= start) & (dataset2['tournament'] == 'FIFA World Cup')]

In [41]:
# Calculate total goals in each match for each dataset
women['total_goals'] = women['home_score'] + women['away_score']
men['total_goals'] = men['home_score'] + men['away_score']


In [42]:

women_goals = women['total_goals']
men_goals = men['total_goals']

In [43]:
# Verify Equal variances
from scipy.stats import levene
stat, p = levene(women_goals, men_goals)
print(f"Levene's test p-value: {p}")

Levene's test p-value: 0.3410985424749826


In [44]:
# Test for normality
from scipy.stats import shapiro, kstest, normaltest

# Shapiro-Wilk Test
women_shapiro = shapiro(women_goals)
men_shapiro = shapiro(men_goals)
print(f"Shapiro-Wilk Test p-values: Women: {women_shapiro.pvalue}, Men: {men_shapiro.pvalue}")


Shapiro-Wilk Test p-values: Women: 3.8905201759850683e-13, Men: 8.894154401688226e-13


While both samples have equal variances, they are not normally distributed. Thus a non-parametric test is more appropriate

In [45]:
from scipy.stats import mannwhitneyu
u_stat, p_val = mannwhitneyu(women_goals, men_goals, alternative='greater')
alpha = 0.10
result = "reject" if p_val < alpha else "fail to reject"
result_dict = {"p_val": p_val, "result": result}
print(result_dict)

{'p_val': 0.005106609825443641, 'result': 'reject'}
